In [1]:
import numpy as np
#from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
import functools
from IPython.display import clear_output

In [2]:
def insert_sample(d,f,fs,samp):        
    if(f in d):
        if(fs in d[f]):
            d[f][fs]['source'].append(samp)
        else:
            d[f][fs] = {'source':[samp]}
    else: ## create new entry
        d[f] = {fs:{'source':[samp]}}


def read_file():
    #read from file, expected frequency and sample array
    #run test, compare expected to processed result
    with open('dataset.txt') as f:
        sample_chunks =[]
        d=dict()
        target_f = 0
        fs = 0
        for line in f:
            line = line.replace(" ","")        
            if(line.lower().find("f=") != -1):
                target_f = (line[line.lower().find("f=")+2:line.find('\n')])
                #if(target_f.isnumeric()):
                #    target_f=np.float64(target_f)
                #else:
                #    target_f = 0
            if(line.lower().find("fs=") != -1):
                fs = np.uint16(line[line.lower().find("fs=")+3:line.find('\n')])

            data_start = line.find('[')
            if(data_start != -1):
                if(fs == 0):
                    print("warning, FS not declared")
                data_chunk = np.array(line[data_start+1:line.find(']')].split(',')).astype(np.int32)
                insert_sample(d,target_f,fs,data_chunk)
    return d    

In [3]:
# total samples not counting auto
def get_total_samples_by_fs(d,fs):
    sample_tot = 0
    for f in d.keys():
        if fs in d[f].keys() and f != 'auto':
            sample_tot += len(d[f][fs]['source'])
    return sample_tot

def get_error_total_of_test(d,fs,test):
    err_tot = 0
    for f in d.keys():
        if fs in d[f].keys():
            err_tot += sum(d[f][fs][test]['error'])
    return err_tot

def list_of_fs(d):
    sorted_fs = []
    for f in d.keys():
        for fs in d[f].keys():
            if fs not in sorted_fs:
                sorted_fs.append(fs)
    sorted_fs.sort(key=int)
    return sorted_fs

def list_of_tests(d):
    tests = []
    first_f = list(d.keys())[0]
    first_fs = list(d[first_f].keys())[0]

    for test in d[first_f][first_fs].keys():
        if test != 'source':
            tests.append(test)    
    return tests

def show_basic_result(d):
    fs_list = list_of_fs(d)
    tests = list_of_tests(d)
    string = ''
    for fs in fs_list:
        samples = get_total_samples_by_fs(d,fs)
        string += '-------------------------\n'
        string += 'fs: '+str(fs)+' total samples: '+str(samples)+'\n'
        string += '|type\t'+'|errors\t|%\t|\n'
        for test in tests:
            errors = get_error_total_of_test(d,fs,test)
            percent_error = (errors/samples)*100
            string += '|'+test+'\t'+str(errors)+'\t|'
            string += '{:3.2f}\t'.format(percent_error)+'|\n'
    return string

In [4]:
def show_source_plot(fs,t,signal,target_f):
    f = 150
    if target_f.isnumeric():
        f = np.int32(target_f)    
    plt.subplot(2,2,1)
    plt.title("Source signal. sample length = %i " %(len(signal)))
    plt.plot(t, signal)
    plt.axhline(0,color='black')
    
    plt.subplot(2,2,2)    
    raw_signal_fft = np.fft.rfft(signal)
    raw_signal_fft_bins = np.fft.rfftfreq(signal.size, d=1/fs)
    max_peak = np.argmax(abs(raw_signal_fft[1:]))+1
    acc = ((fs/2)/(len(signal)/2))
    plt.title("FFT acc +/- %i Hz | dominant f %i Hz" %(acc,raw_signal_fft_bins[max_peak]))
    plt.axvline(raw_signal_fft_bins[max_peak],color="green")
    plt.plot(raw_signal_fft_bins,np.abs(raw_signal_fft))
    plt.xlim(0, f*5) #show range up to 4th harmonic    
    plt.tight_layout()
    plt.show()    

In [5]:
def show_result_plot(fs,t,result,target_f,t_idx):
    plt.subplot(2,2,1)
    plt.plot(t,result)
    plt.plot(t[t_idx],result[t_idx],'ro')
    plt.title("f[%i]=%.2f Hz | T=%.3f ms" %(t_idx,fs/t_idx,(t_idx*1000)/fs))    
    
    t_adj = get_interpolation(result,t_idx)
    plt.subplot(2,2,2)
    window = 5
    plt.plot(t[t_idx-window:t_idx+window+1],result[t_idx-window:t_idx+window+1])
    
    plt.plot(t_adj/fs,result[t_idx],"bo",label = 'adjusted center')
    plt.plot(t[t_idx+1],result[t_idx+1],"go",label = 'center +1')
    plt.plot(t[t_idx],result[t_idx],"ro",label='center')
    plt.plot(t[t_idx-1],result[t_idx-1],"go",label = 'center -1')
    plt.legend()
    plt.title("f[%.2f]=%.2f Hz | T=%.3f ms" %(t_adj,fs/t_adj,(t_adj*1000)/fs))
    plt.tight_layout()
    plt.show()

In [6]:
def acf(x):
    r = np.zeros(len(x))
    for k in range(len(x)):
        for j in range(len(x)-k):
            r[k] += (x[j]*x[j+k]) #acf
        #r[k] = r[k]/(len(x)+3*k)
        r[k] = r[k]/(len(x)+k)
        #r[k] = r[k]/(len(x))
    return r

In [7]:
def amdf(x):
    r = np.zeros(len(x))
    for k in range(0,len(x)):
        for j in range(len(x)-k):
            r[k] += abs(x[j]-x[j+k])
        #r[k] = r[k]/(len(x)-k)
        #r[k] = r[k]/len(x)
        #r[k] = r[k]+20*k
    return r

In [8]:
def get_idx(x,fs,corr_type):
    T_MIN = fs/365
    T_MAX = fs/65
    target_val = 0
    t_idx = -1
    
    if corr_type == "amdf":
        target_val = np.iinfo(np.int16).max
        
    for i in range(len(x)):
        if( i  > T_MIN and i < T_MAX):
            if corr_type == "acf":                
                if x[i] > target_val:
                    target_val = x[i]
                    t_idx = i 
            elif corr_type == "amdf":
                if x[i] < target_val:
                    target_val = x[i]
                    t_idx = i
    return t_idx
        

In [9]:
def get_interpolation(x,t_idx):
    p_adj = ((x[t_idx+1] - x[t_idx-1])/(2*(2*x[t_idx] - x[t_idx+1] - x[t_idx-1])))
    return t_idx+p_adj

In [10]:
def get_results(d,target_f,fs,corr_type):
    res_d={'res':[],'idx':[],'f':[],'error':[]}

    for i in range(len(d[target_f][fs]['source'])):
        signal = d[target_f][fs]['source'][i]
        res_d["res"].append(eval(corr_type)(signal))
        res_d["idx"].append(get_idx(res_d["res"][i],fs,corr_type))
        res_d["f"].append(fs/get_interpolation(res_d["res"][i],res_d["idx"][i]))
        if target_f.isnumeric():
            f = np.int32(target_f)
            res_d["error"].append(abs(f - res_d["f"][i]) > 2)
        
    return res_d

In [11]:
#run all tests
def run_tests(d):    
    for f in d.keys():
        for fs in d[f].keys():
            acf_res = get_results(d,f,fs,'acf')            
            d[f][fs]['acf'] = acf_res
            amdf_res = get_results(d,f,fs,'amdf')            
            d[f][fs]['amdf'] = amdf_res

d = read_file()
run_tests(d)
text_result = show_basic_result(d)

In [15]:
freq_list = d.keys()

f_select = widgets.Dropdown(options=freq_list,description = 'Freq',layout={'width': 'max-content'})
fs_select = widgets.Dropdown(options=d[f_select.value].keys(),description='FS',layout={'width': 'max-content'})
test_list = list_of_tests(d)
test_select = widgets.Dropdown(options=test_list,description='Test',layout={'width': 'max-content'})

button_grid =widgets.Output(layout={'width':'auto','border': '1px solid black'})
basic_info_out =widgets.Output(layout={'width':'auto','border': '1px solid black'})
out =widgets.Output(layout={'width':'auto','border': '1px solid black'})

source_options = widgets.VBox([f_select,fs_select])
test_options = widgets.VBox([test_select])
options_box = widgets.HBox([source_options,test_options])
ui = widgets.VBox([options_box,button_grid])
selected = 0

def show_plot(f,fs,test,sample_idx):
    with out:
        clear_output()
        signal_data = d[f][fs]['source'][sample_idx]
        s_total = len(d[f][fs]['source'])
        l = len(signal_data)/fs 
        t = np.arange(0,l,1.0/fs) #x axis time scale
        
        result = d[f][fs][test]['res'][sample_idx]
        t_idx = d[f][fs][test]['idx'][sample_idx]
        err = sum(d[f][fs][test]['error'])
    
        print("f",f,"fs",fs,test,"sample",sample_idx)
        print("Num of samples in test",s_total)
        print("Errors\t",err)
        print("Error %\t {:3.2f}".format((err/s_total)*100))
        # print("Errors", d[f][fs][test]["error"])
        
        plt.rcParams['figure.figsize'] = [10, 5]
        show_source_plot(fs,t,signal_data,f)        
        show_result_plot(fs,t,result,f,t_idx)    

def sample_selected(b,f,fs,test):
    global selected 
    selected = int(b.description)
    show_plot(f,fs,test,selected)

def update_buttons(f,fs,test):            
    error_list = []
    if(f.isnumeric()):
        error_list = (d[f][fs][test]['error'])
    data = d[f][fs][test]['res']
    list_len = len(data)    
    button_layout = widgets.Layout(height='auto', width='auto')
    grid_layout=widgets.Layout(grid_template_columns='auto auto auto auto auto')
    buttons = []
    for i in range(list_len):
        button_style = ''
        if f.isnumeric():
            button_style = "danger"*int(error_list[i]) or "success"
        buttons.append(widgets.Button(description=str(i), layout=button_layout,button_style=button_style))

    for i in range(list_len):
        buttons[i].on_click(functools.partial(sample_selected,f=f,fs=fs,test=test)) 
    grid = widgets.GridBox(children=buttons,layout=grid_layout)
    return grid

def change_event(*args):
    global selected
    fs_select.options = d[f_select.value].keys()
    sample_max = len(d[f_select.value][fs_select.value]['source'])
    if selected > sample_max: selected = 0
    btn_grid = update_buttons(f_select.value,fs_select.value,test_select.value)
    #update button grid
    with button_grid:
        clear_output()
        display(btn_grid)
    show_plot(f_select.value,fs_select.value,test_select.value,selected)
        
f_select.observe(change_event, 'value')
fs_select.observe(change_event, 'value')
test_select.observe(change_event, 'value')

# init
with button_grid:
    btn_grid = update_buttons(f_select.value,fs_select.value,test_select.value)
    display(btn_grid)
with basic_info_out:
    print(text_result)
with out:
    show_plot(f_select.value,fs_select.value,test_select.value,selected)

widgets.HBox([widgets.VBox([ui,basic_info_out]),out])
